In [75]:
import pandas as pd
import numpy as np

In [76]:
data = pd.read_csv('./data/user.csv',encoding='CP949')

In [77]:
data

,userId,wineCode,rating
0,beDeveloper,170309,5.0
1,beDeveloper,170308,5.0
2,beDeveloper,170270,3.0
3,beDeveloper,170334,NaN
4,beDeveloper,170310,NaN
5,pizzalove,170309,1.0
6,pizzalove,170308,NaN
7,pizzalove,170270,1.0
8,pizzalove,170334,3.0
9,pizzalove,170310,5.0


In [78]:
ratings = data.pivot_table(index='userId',columns='wineCode',values='rating')
ratings

wineCode,170212,170214,170216,170270,170308,170309,170310,170334
userId,,,,,,,,
beDeveloper,5.0,NaN,3.0,3.0,5.0,5.0,NaN,NaN
deepbrain,2.0,NaN,3.0,5.0,NaN,1.0,NaN,5.0
groot,2.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN
groot2,4.0,NaN,4.0,1.0,4.0,5.0,NaN,1.0
pizzalove,3.0,4.0,1.0,1.0,NaN,1.0,5.0,3.0
rapperDeveloper,5.0,2.0,NaN,5.0,3.0,NaN,5.0,5.0


In [79]:
ratings.index

Index(['beDeveloper', 'deepbrain', 'groot', 'groot2', 'pizzalove',
       'rapperDeveloper'],
      dtype='object', name='userId')

In [80]:
from itertools import product
index_pair = list(product(ratings.index,repeat=2))
index_pair

[('beDeveloper', 'beDeveloper'),
 ('beDeveloper', 'deepbrain'),
 ('beDeveloper', 'groot'),
 ('beDeveloper', 'groot2'),
 ('beDeveloper', 'pizzalove'),
 ('beDeveloper', 'rapperDeveloper'),
 ('deepbrain', 'beDeveloper'),
 ('deepbrain', 'deepbrain'),
 ('deepbrain', 'groot'),
 ('deepbrain', 'groot2'),
 ('deepbrain', 'pizzalove'),
 ('deepbrain', 'rapperDeveloper'),
 ('groot', 'beDeveloper'),
 ('groot', 'deepbrain'),
 ('groot', 'groot'),
 ('groot', 'groot2'),
 ('groot', 'pizzalove'),
 ('groot', 'rapperDeveloper'),
 ('groot2', 'beDeveloper'),
 ('groot2', 'deepbrain'),
 ('groot2', 'groot'),
 ('groot2', 'groot2'),
 ('groot2', 'pizzalove'),
 ('groot2', 'rapperDeveloper'),
 ('pizzalove', 'beDeveloper'),
 ('pizzalove', 'deepbrain'),
 ('pizzalove', 'groot'),
 ('pizzalove', 'groot2'),
 ('pizzalove', 'pizzalove'),
 ('pizzalove', 'rapperDeveloper'),
 ('rapperDeveloper', 'beDeveloper'),
 ('rapperDeveloper', 'deepbrain'),
 ('rapperDeveloper', 'groot'),
 ('rapperDeveloper', 'groot2'),
 ('rapperDeveloper',

In [81]:
def get_cosine_similarity(u1,u2):
    mask=np.isfinite(u1) & np.isfinite(u2)
    u=u1[mask]
    v=u2[mask]
    uvdot=(u*v).sum()
    norm1=(u**2).sum()
    norm2=(v**2).sum()
    score = uvdot/np.sqrt(norm1*norm2)
    return score
    

In [82]:
sample = np.array([3,np.nan,4,5])
sample2 = np.array([np.nan,2,1,3])
get_cosine_similarity(sample,sample2)

0.9383431168171101

In [83]:
def get_cosine_similarity_table(ratings):
    index_pair = list(product(ratings.index,repeat=2))
    similarity_list=[]
    for u1,u2 in index_pair:
        u,v =ratings.loc[u1],ratings.loc[u2]
        score=get_cosine_similarity(u,v)
        similarity= { 'u':u1,
                      'v':u2,
                      'score':score,}
        similarity_list.append(similarity)
    
    similarity_list=pd.DataFrame(similarity_list)
    similarity_table=pd.pivot_table(similarity_list,index="u",columns="v",values="score")
    return similarity_table

In [84]:
similarity_table=get_cosine_similarity_table(ratings)
similarity_table

v,beDeveloper,deepbrain,groot,groot2,pizzalove,rapperDeveloper
u,,,,,,
beDeveloper,1.000000,0.757317,0.953206,0.964345,0.910182,0.932203
deepbrain,0.757317,1.000000,0.786796,0.569576,0.818317,0.942809
groot,0.953206,0.786796,1.000000,0.904194,0.813733,0.948504
groot2,0.964345,0.569576,0.904194,1.000000,0.710239,0.785905
pizzalove,0.910182,0.818317,0.813733,0.710239,1.000000,0.860828
rapperDeveloper,0.932203,0.942809,0.948504,0.785905,0.860828,1.000000


In [85]:
ratings.T

userId,beDeveloper,deepbrain,groot,groot2,pizzalove,rapperDeveloper
wineCode,,,,,,
170212,5.0,2.0,2.0,4.0,3.0,5.0
170214,NaN,NaN,NaN,NaN,4.0,2.0
170216,3.0,3.0,1.0,4.0,1.0,NaN
170270,3.0,5.0,1.0,1.0,1.0,5.0
170308,5.0,NaN,1.0,4.0,NaN,3.0
170309,5.0,1.0,1.0,5.0,1.0,NaN
170310,NaN,NaN,1.0,NaN,5.0,5.0
170334,NaN,5.0,NaN,1.0,3.0,5.0


In [86]:
wine_similarity_table=get_cosine_similarity_table(ratings.T)
wine_similarity_table

v,170212,170214,170216,170270,170308,170309,170310,170334
u,,,,,,,,
170212,1.000000,0.843661,0.919145,0.822464,0.970720,0.937889,0.954053,0.843274
170214,0.843661,1.000000,1.000000,0.613941,1.000000,1.000000,0.948683,0.843661
170216,0.919145,1.000000,1.000000,0.821995,0.968364,0.915737,0.832050,0.729293
170270,0.822464,0.613941,0.821995,1.000000,0.816830,0.609712,0.835400,0.966755
170308,0.970720,1.000000,0.968364,0.816830,1.000000,0.993912,0.992278,0.745241
170309,0.937889,1.000000,0.915737,0.609712,0.993912,1.000000,0.832050,0.422890
170310,0.954053,0.948683,0.832050,0.835400,0.992278,0.832050,1.000000,0.970143
170334,0.843274,0.843661,0.729293,0.966755,0.745241,0.422890,0.970143,1.000000


In [87]:
neighbors_ratings = ratings[170212].drop(index="beDeveloper")
print(neighbors_ratings)

userId
deepbrain          2.0
groot              2.0
groot2             4.0
pizzalove          3.0
rapperDeveloper    5.0
Name: 170212, dtype: float64


In [88]:
def predict_rating(userId,wineCode):
    neighbors_ratings = ratings[wineCode].drop(index=userId)
#     print(neighbors_ratings)
    neighbors_similarity = similarity_table[userId].drop(index=userId)
#     print(neighbors_similarity)
    nominator=(neighbors_ratings * neighbors_similarity).sum()
    denominator=neighbors_similarity.sum()
    score=nominator/denominator
    return score

In [89]:
def predict_rating_table(ratings):
    rating_combinations=list(product(ratings.index,ratings.columns))
    rating_list=[]
    for userId,wineCode in rating_combinations:
        rating=predict_rating(userId,wineCode)
        rating_predict={
            'user':userId, 'wine':wineCode, 'rating':rating
        }
        rating_list.append(rating_predict)
    
    rating_list=pd.DataFrame(rating_list)
    rating_table=rating_list.pivot_table(index='user',columns='wine',values='rating')
    rating_table=rating_table[ratings.isnull()]
    return rating_table


In [90]:
rating_table=predict_rating_table(ratings)
rating_table

wine,170212,170214,170216,170270,170308,170309,170310,170334
user,,,,,,,,
beDeveloper,NaN,1.218690,NaN,NaN,NaN,NaN,2.250290,2.688025
deepbrain,NaN,1.331389,NaN,NaN,2.498213,NaN,2.475583,NaN
groot,NaN,1.169186,NaN,NaN,NaN,NaN,NaN,2.728259
groot2,NaN,1.121625,NaN,NaN,NaN,NaN,2.131256,NaN
pizzalove,NaN,NaN,NaN,NaN,2.622732,NaN,NaN,NaN
rapperDeveloper,NaN,NaN,2.366308,NaN,NaN,2.537371,NaN,NaN


In [91]:
def predict_wine(userId,k=1):
    prediction_list = rating_table.loc[userId].sort_values(ascending=False)
    prediction_list=prediction_list.head(k).index
    return prediction_list


In [92]:
pd.DataFrame(predict_wine("groot2",k=3))

,wine
0,170310
1,170214
2,170212


In [93]:
predict_wine("groot2",k=3)

Int64Index([170310, 170214, 170212], dtype='int64', name='wine')

In [94]:
def predict_user(wineCode,k=1):
    prediction_list=rating_table[wineCode].sort_values(ascending=False)
    print(prediction_list)
    prediction_list=prediction_list.head(k).index
    return prediction_list


In [95]:
predict_user(170310)

user
deepbrain          2.475583
beDeveloper        2.250290
groot2             2.131256
groot                   NaN
pizzalove               NaN
rapperDeveloper         NaN
Name: 170310, dtype: float64


Index(['deepbrain'], dtype='object', name='user')

In [96]:
def get_pearson_correlation(u, v, epsilon=0.0001):
    mask = (np.isfinite(u) & np.isfinite(v))
    
    u, v = u[mask], v[mask]
    
    u_dev, v_dev = u-u.mean(), v-v.mean()
    u_var, v_var = (u_dev**2).sum(), (v_dev**2).sum()
    
    
    nominator = np.dot(u_dev, v_dev)
    denominator = np.sqrt(u_var * v_var)
    
    pearson_correlation = nominator / (denominator + epsilon)
    
    return pearson_correlation    

In [97]:
# 분모가 0이 되는 경우를 제외하기 위해 최종적으로 분모에 epsilon을 넣어준다.
def get_pearson_correlation_table(ratings, epsilon = 0.0001):
    
    index_pair = list(product(ratings.index, repeat=2))
    similarity_list = []

    for u_name, v_name in index_pair:
        
        u = ratings.loc[u_name]
        v = ratings.loc[v_name]
    
        score = get_pearson_correlation(u, v)

        similarity = {
            'u': u_name,
            'v': v_name,
            'score': score
        }
        
        similarity_list.append(similarity)  
    
    similarity_list = pd.DataFrame(similarity_list)
    similarity_table= pd.pivot_table(similarity_list, index="u", columns="v", values="score")

    return similarity_table

In [98]:
get_pearson_correlation_table(ratings)

v,beDeveloper,deepbrain,groot,groot2,pizzalove,rapperDeveloper
u,,,,,,
beDeveloper,0.999979,-0.845140,0.408227,0.662112,0.577334,-0.499981
deepbrain,-0.845140,0.999992,-0.292759,-0.971116,0.153091,0.000000
groot,0.408227,-0.292759,0.999880,0.147437,0.249992,0.333311
groot2,0.662112,-0.971116,0.147437,0.999993,-0.243972,-0.577339
pizzalove,0.577334,0.153091,0.249992,-0.243972,0.999994,-0.301508
rapperDeveloper,-0.499981,0.000000,0.333311,-0.577339,-0.301508,0.999989


In [99]:

def get_nearest_neighbors(userId, wineCode, ratings, similarity, k):

    max_neighbors_num = similarity.shape[0] - 1
    k = min(k, max_neighbors_num)
    
    neighbors_similarity = similarity[userId].drop(index=userId)
    neighbors_ratings = ratings[wineCode].drop(index=userId)
    
    # 해당 상품에 평점을 준 적이 없으면 제외
    neighbors_similarity = neighbors_similarity[neighbors_ratings.notnull()]
    neighbors_ratings = neighbors_ratings[neighbors_ratings.notnull()]

    if neighbors_ratings.shape[0] == 0:
        return neighbors_ratings.index
        
    neighbors_similarity = neighbors_similarity.sort_values(ascending=False)
    candidates = neighbors_similarity[neighbors_similarity > 0]

    if (candidates.shape[0]) == 0:
        nearest_neighbors = neighbors_similarity.head(1)
    else:
        nearest_neighbors = candidates.head(k)

    return nearest_neighbors.index

In [100]:
pearson_correlation = get_pearson_correlation_table(ratings)
pd.DataFrame(get_nearest_neighbors('deepbrain', 170310, ratings, pearson_correlation, k=2))

,u
0,pizzalove


In [101]:
# 특정 사용자의 상품의 평점을 예측

def user_rating_predict(userId, wineCode, ratings, similarity, k, epsilon=0.0001):

    nearest_neighbors = get_nearest_neighbors(userId, wineCode, ratings, similarity, k)
    
    if len(nearest_neighbors) == 0:
        return np.nan
    
    nn_similarity = similarity.loc[userId, nearest_neighbors]
    nn_ratings = ratings.loc[nearest_neighbors, wineCode]
    
    # 이웃들이 예측하려는 상품에 내린 평점의 평균.
    neighbors_rating_mean = nn_ratings.mean()
    # 예측하려는 사용자가 해당 해당상품에 내린 평점의 평균.  
    u_rating_mean = ratings.loc[userId].mean()
    

    nominator = np.dot(nn_similarity,(nn_ratings-neighbors_rating_mean))
    denominator = nn_similarity.sum()
    prediction = u_rating_mean + nominator / (denominator + epsilon)
    
    return prediction

In [102]:

pearson_correlation = get_pearson_correlation_table(ratings)

user_rating_predict("groot", 170310, ratings, pearson_correlation, k=3)

1.1666666666666667

In [103]:
# 사용자 기반, 상품 기반 기법 모두 가능
def ratings_predict(ratings, kind='user', k=2, epsilon=0.0001):
    
    if kind == 'wine':
        ratings = ratings.T
    
    similarity = get_pearson_correlation_table(ratings, epsilon)
    prediction_list = []

    
    for u_name, v_name in list(product(ratings.index, ratings.columns)):
        rating = user_rating_predict(u_name, v_name, ratings, similarity, k, epsilon)
        prediction = {
            'u' : u_name,
            'v' : v_name,
            "rating" : rating
        }
        prediction_list.append(prediction)
        
    result = pd.DataFrame(prediction_list).pivot_table(index='u', columns='v', values='rating')
    result = result[ratings.isnull()]
        
    if kind == 'wine':
        return result.T

    return result

In [104]:
ratings_predict(ratings, 'user', k=2, epsilon=0.0001)

v,170212,170214,170216,170270,170308,170309,170310,170334
u,,,,,,,,
beDeveloper,NaN,4.200000,NaN,NaN,NaN,NaN,4.543132,4.131605
deepbrain,NaN,3.200000,NaN,NaN,3.200000,NaN,3.200000,NaN
groot,NaN,1.023851,NaN,NaN,NaN,NaN,NaN,1.309482
groot2,NaN,3.166667,NaN,NaN,NaN,NaN,3.166667,NaN
pizzalove,NaN,NaN,NaN,NaN,3.362657,NaN,NaN,NaN
rapperDeveloper,NaN,NaN,4.166667,NaN,NaN,4.166667,NaN,NaN


In [106]:
a = ratings_predict(ratings, 'wine', k=2, epsilon=0.0001)